In [1]:
from zoegas.reader import Dictionary, dictionary_name
import re
from zoegas.constants import pos_verbose
from cltk.corpus.swadesh import swadesh_old_norse

In [2]:
d = Dictionary(dictionary_name)

In [3]:
# def extract_parts(text):
    
def extract_category(text):
    p = re.compile(r"\(.+?\), (?P<category>\w+)\.")
    m = p.search(text)
    return m.group("category")

def extract_subentries(text):
    subentries = [subentry for subentry in re.split(r'[IVX]+\)', text) if subentry.strip()]
    return subentries

def extract_group_in_parentheses(text):
    p = re.compile(r'\((?P<first>.+?)\)')
    m = p.search(text)
    if m is not None:
        print(m.group("first"))
        return m.group("first")
    else:
        return text
        

In [4]:
word = "abbadis"
entry = d.find(word)

In [5]:
description = entry.description.strip()
print(description)
p = re.compile(r'\((?P<subcategory>\w+)\. -(?P<ending>\w+)\), (?P<category>\w+)\. ')
m = p.search(description)
print(m.group("subcategory"))
print(m.group("ending"))
print(m.group("category"))


(pl. -ar), f. abbess.
pl
ar
f


In [6]:
description

'(pl. -ar), f. abbess.'

In [7]:
print(pos_verbose.keys())
print(pos_verbose)
def strip_ending(text: str, word_category: str) -> str:
    if word_category in ["m", "f", "n"]:
        if text.endswith(text, "i"):
            return text[:-1]
        elif text.endswith():
            return text
    elif word_category in ["v. refl."]:
        if text.endswith(text, "st"):
            return text[:-2]
    return text

dict_keys(['a.', 'acc.', 'adv.', 'card. numb.', 'compar.', 'conj.', 'dat.', 'def. art.', 'dem. pron.', 'f.', 'for.', 'fem.', 'gen.', 'imperat.', 'impers.', 'indecl.', 'indef. pron.', 'infin.', 'int. pron.', 'interj.', 'm.', 'masc.', 'n.', 'neut.', 'nom.', 'ord. numb.', 'pers. pron.', 'pl.', 'poss. pron.', 'pp.', 'pr. p.', 'prep.', 'pron.', 'refl.', 'refl. pron.', 'rel. pron.', 'sing.', 'superl.', 'v.', 'v. refl.'])
defaultdict(<class 'str'>, {'a.': 'adjective', 'acc.': 'accusative', 'adv.': 'adverb', 'card. numb.': 'cardinal number', 'compar.': 'comparative', 'conj.': 'conjunction', 'dat.': 'dative', 'def. art.': 'definite article', 'dem. pron.': 'demonstrative pronoun', 'f.': 'feminine noun', 'for.': 'e', 'fem.': 'feminine', 'gen.': 'genitive', 'imperat.': 'imperative', 'impers.': 'impersonal', 'indecl.': 'indeclinable', 'indef. pron.': 'indefinite pronoun', 'infin.': 'infinitive', 'int. pron.': 'interrogative pronoun', 'interj.': 'interjection', 'm.': 'masculine noun', 'masc.': 'masc

In [8]:
word = "segja"
entry = d.find(word)

In [9]:
description = entry.description.strip()
# print(description)
first_group = extract_group_in_parentheses(description)
category = extract_category(description)

def extract_weak_verb_forms(text):
    p = re.compile(r'\((?P<sfg3en>\w+), (?P<sfg3et>\w+), (?P<stken>\w+)\)')
    m = p.search(description)
#     print(m.group("sfg3en"))
#     print(m.group("sfg3et"))
#     print(m.group("stken"))
    return {"sfg3en": m.group("sfg3en"), "sgg3et": m.group("sfg3et"), "stken": m.group("stken")}
print(category)
weak_verb_forms = extract_weak_verb_forms(first_group)
print(weak_verb_forms)

segi, sagða, sagðr
v
{'sfg3en': 'segi', 'sgg3et': 'sagða', 'stken': 'sagðr'}


In [10]:
word = "jötunn"
entry = d.find(word)
description = entry.description.strip()
print(description)



def extract_sub_element(text):
#     element_extractor_ending = re.compile(r'(?P<subcategory>\w+)\. -(?P<ending>\w+)')
    element_extractor = re.compile(r'(?P<subcategory>\w+)\. (?P<form>[\w\- ]+)')
    m = element_extractor.search(text)
    if m is not None:
        l = (m.group("subcategory"), analyse_given_form(m.group("form")))
#         print(m.group("subcategory"))
#         print(m.group("form"))
    else:
#         print("text : "+text)
        form_extractor = re.compile(r'(?P<form>[\w\- ]+)')
        m = form_extractor.search(text)
        l = [m.group("form")]
    return l
        
        
def extract_list_from_parentheses(text):
    l = []
    singular_plural = text.split(";")
    if len(singular_plural) > 1:
        for i in singular_plural:
            for element in i.split(","):
                l.append(extract_sub_element(element))
    else:
        
        forms = text.replace(",", "").split(" ")
#         print("forms:", forms)
        if "later" in forms:
            before, after = text.split("later")
            l_before = []
            l_after = []
            for element in before.split(","):
                if element.strip():
                    l_before.append(extract_sub_element(element))
            for element in after.split(","):
                if element.strip():
                    l_after.append(extract_sub_element(element))
            l = {"before": l_before, "after": l_after}
        else:
#             print(text)
            for element in text.split(","):
                if element.strip():
                    l.append(extract_sub_element(element))
    return l


def analyse_given_form(text):
    l = []
    text = text.replace(",", "")
    words = text.split(" ")
    
#     print(text)
    if "and" in words:
        for i in text.split("and"):
            print(i)
            l.append({"complete": i.strip()})
    elif "later," in words:
        before_after = text.split("later")
        before = before_after[0]
        nee, nfn = before.split(" ")
        nee2, nfn2 = before_after[1].split(" ")
        l.append({})
        
    elif text[0] == "-":
        l.append({"ending": text[0:]})
    else:
        l.append({"complete": text})
    return l
    
    
first_group = extract_group_in_parentheses(description)
l = extract_list_from_parentheses(first_group)

print(l)



(gen. -s, dat. jötni; pl. jötnar), m. giant; jötna synir, sons of giants
gen. -s, dat. jötni; pl. jötnar
[('gen', [{'ending': '-s'}]), ('dat', [{'complete': 'jötni'}]), ('pl', [{'complete': 'jötnar'}])]


In [11]:
word = "áss"
entry = d.find(word)
description = entry.description.strip()
print(description)
first_group = extract_group_in_parentheses(description)
l = extract_list_from_parentheses(first_group)
print(l)

I)

(gen. áss and ásar; pl. æsir, acc. æsi and ásu), m. one of the old heathen gods in general, or esp. one of the older branch, in opp. to the younger ones (the Vanir).

II)

(gen. áss, pl. ásar), m.

1) a thick pole, main beam (in a house);

2) in a ship, yard of a sail (beitiáss);

3) rocky ridge.
gen. áss and ásar; pl. æsir, acc. æsi and ásu
áss 
 ásar
æsi 
 ásu
[('gen', [{'complete': 'áss'}, {'complete': 'ásar'}]), ('pl', [{'complete': 'æsir'}]), ('acc', [{'complete': 'æsi'}, {'complete': 'ásu'}])]


In [12]:
word = "aðili"
entry = d.find(word)

In [13]:
description = entry.description.strip()
print(description)

first_group = extract_group_in_parentheses(description)
l = extract_list_from_parentheses(first_group)
print(l)

(-ja, -jar, later -a, -ar), m. chief defendant (varnar~) or prosecutor (sóknar~, sakar~).
-ja, -jar, later -a, -ar
{'before': [['-ja'], [' -jar']], 'after': [[' -a'], [' -ar']]}


In [14]:
for word in swadesh_old_norse:
    print(word)
    if word is None:
        continue
    entry = d.find(word)
    if entry is not None:
        description = entry.description.strip()
#         print(description)
        first_group = extract_group_in_parentheses(description)
        if first_group == description:
            l = extract_subentries(description)
            print(l)
        else:
            l = extract_list_from_parentheses(first_group)
            print(l)
    else:
        print(word.capitalize() + " is not in the dictionary")

ek
[' pers. pron. I;\n\nin poetry and old prose a pronominal k is suffixed to the verb, emk = em ek, vask = vas ek; sák = sá ek; mundak = munda ek; even if preceded by ek: ek sék, ek sitk; a preceding g becomes by assimilation k, hykk = hygg ek; the pronominal k is inserted between the suffixed negative, -a or -at, and the verb, sáka = sá ek-a, I saw not; veitka = veit ek-a, I know not.\n\n', ' from aka.']
þú
gen. þín, dat. þér, acc. þik; pl. þér or ér, gen. yðar, dat. and acc. yðr

 acc
[('gen', [{'complete': 'þín'}]), ('dat', [{'complete': 'þér'}]), ('acc', [{'complete': 'þik'}]), ('pl', [{'complete': 'þér or ér'}]), ('gen', [{'complete': 'yðar'}]), ('dat', [{'complete': ''}, {'complete': 'acc'}])]
hann
['pers. pron. he.']
vér
Vér is not in the dictionary
þér
Þér is not in the dictionary
þeir
þær, þau
[['þær'], [' þau']]
sjá, þessi
Sjá, þessi is not in the dictionary
sá
sú, þat
[['sú'], [' þat']]
hér
margir þeir er ~ koma
[['margir þeir er ']]
þar
~ var fjölmenni mikit
[[' var fjölme